In [15]:
from os.path import join as opj
import os
import pandas as pd
from tqdm.auto import tqdm
from traffic.data import airports, aircraft
from traffic.core import Traffic, Flight
import plotly.express as px
import plotly.graph_objects as go

import scipy.stats as st
from datetime import timedelta

from scipy.signal import argrelextrema
import numpy as np

In [16]:
width = 1000
height = width / 1.618
(width, height)

(1000, 618.0469715698392)

# Data loading


In [17]:
df_landings = pd.read_parquet(opj("data", "osn23_landings_merged.parquet.gzip"))
print(f"{len(df_landings)} landings are in the dataset")
df_landings

7984254 landings are in the dataset


,flight_id,callsign,icao24,landing_time,registration,typecode,icaoaircrafttype,airport,country,attempt_times,ILS,n_attempts,GoA,market_segment,AC_CLASS,AP_C_RWY,C40_CROSS_TIME,C40_BEARING,C40_CROSS_LAT,C40_CROSS_LON
1,EZY3092_21625,EZY3092,440031,2019-01-01 00:15:21+00:00,,,,EGSS,United Kingdom,[2019-01-01T00:15:21.000000],[22],1,False,None,None,None,NaT,NaN,NaN,NaN
2,PGT77F_32108,PGT77F,4b8e46,2019-01-01 00:23:58+00:00,None,None,None,LTFJ,Turkey,[2019-01-01T00:23:58.000000],[06L],1,False,None,None,None,NaT,NaN,NaN,NaN
3,JTG260_22155,JTG260,45ab49,2019-01-01 00:28:15+00:00,None,None,None,EKCH,Denmark,[2019-01-01T00:28:15.000000],[22L],1,False,None,None,None,NaT,NaN,NaN,NaN
5,TFL554_98026,TFL554,484ad0,2019-01-01 00:56:00+00:00,PHTFB,B738,L2J,EHAM,Netherlands,[2019-01-01T00:55:24.000000],[18R],1,False,None,MJ,18R,2019-01-01 00:40:34+00:00,232.246139,51.895833,3.908056
6,VKG799_43226,VKG799,45d968,2019-01-01 02:34:54+00:00,OYVKH,A333,L2J,EKCH,Denmark,[2019-01-01T02:34:54.000000],[22L],1,False,Non-Scheduled,H,22L,2019-01-01 02:21:40+00:00,46.598471,56.073056,13.523889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8163976,RYR593Y_66483,RYR593Y,4ca9ec,2023-05-30 23:49:56+00:00,EIEVO,B738,L2J,LEMD,Spain,[2023-05-30T23:49:56.000000],[32R],1,False,None,None,None,NaT,NaN,NaN,NaN
8163977,EXS74QM_1825,EXS74QM,406a62,2023-05-30 23:50:27+00:00,GGDFX,B738,L2J,EGSS,United Kingdom,[2023-05-30T23:50:27.000000],[04],1,False,None,None,None,NaT,NaN,NaN,NaN
8163978,BCS460_25089,BCS460,4cac6d,2023-05-30 23:50:42+00:00,EISTW,B734,L2J,LIMC,Italy,[2023-05-30T23:50:17.000000],[35L],1,False,None,MJ,35L,2023-05-30 23:35:33+00:00,342.087208,46.264722,8.426111
8163979,WMT95FV_41476,WMT95FV,4d242c,2023-05-30 23:58:45+00:00,None,None,None,LIRF,Italy,[2023-05-30T23:58:45.000000],[16R],1,False,None,None,None,NaT,NaN,NaN,NaN


### Filtering out general aviaiton / helicopters ...


In [18]:
valid_types = ["L2J", "L4J", "L2T", "L3J", "L4T", "L1T", "L3T", "L1J"]
df_landings = df_landings.query("icaoaircrafttype in @valid_types")
l_invalid = ("FCK", "GAF", "PPU", "CFL", "ECK", "CALIBRAT", "PHLAB", "CALIBRA", "ENF")
df_landings = df_landings.query("not callsign.str.startswith(@l_invalid)")
df_landings = df_landings.query(
    "n_attempts<4"
)  # if there are more than 4 attempts it is most likely a trainning flight
print(
    f"Among {len(df_landings)} commercial aviation landings, {len(df_landings.query('GoA'))} are GoA"
)

Among 6565318 commercial aviation landings, 20179 are GoA


In [19]:
# check the percentage of rows that have a C40_BEARING value
print(
    f"{len(df_landings.query('C40_BEARING.notnull()'))/len(df_landings):.2%} of the landings have a C40_BEARING value"
)  # make the count per airport
for airport, group in df_landings.groupby("airport"):
    print(f"{airport}: {len(group.query('C40_BEARING.notnull()'))/len(group):.2%}")

92.04% of the landings have a C40_BEARING value
EDDB: 96.02%
EDDF: 98.92%
EDDM: 95.97%
EGCC: 51.54%
EGLL: 97.49%
EGSS: 96.43%
EHAM: 97.88%
EIDW: 98.56%
EKCH: 95.57%
EPWA: 97.45%
ESSA: 97.73%
LEMD: 96.75%
LFPG: 97.84%
LFPO: 98.06%
LIMC: 96.04%
LIRF: 97.70%
LOWW: 97.42%
LSGG: 95.43%
LSZH: 95.27%
LTFJ: 0.00%


### Where are the landings?


In [20]:
fig = px.sunburst(
    df_landings.groupby(["airport"])
    .agg({"flight_id": "count", "country": "first"})
    .reset_index(),
    path=["country", "airport"],
    values="flight_id",
    labels={"country": "Country"},
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.data[0].textinfo = "label+percent entry+value"
fig.show()
fig.write_image("figures/landings_country_airport.pdf")

# Analysis


## High Level


### GoA Rate per market segment


In [21]:
# Define the aggregation functions
agg_functions = {"GoA": ["mean", "sum"], "flight_id": "count"}

# Aggregate the data by market segment
df_agg = df_landings.groupby("market_segment").agg(agg_functions)

# Rename the columns
df_agg.columns = ["ga_rate", "n_ga", "n_landings"]

# Compute the Clopper-Pearson interval for the GoA rate
alpha = 0.05
n = df_agg["n_landings"]
k = df_agg["n_ga"]
lower_ci, upper_ci = st.beta.interval(1 - alpha, k, n - k + 1)
df_agg["ga_rate_lower"] = lower_ci * 1000
df_agg["ga_rate_upper"] = upper_ci * 1000
df_agg["ga_rate"] = df_agg["ga_rate"] * 1000

# Compute the lower and upper bounds of the confidence interval
df_agg["ga_rate_lb"] = df_agg["ga_rate"] - df_agg["ga_rate_lower"]
df_agg["ga_rate_ub"] = df_agg["ga_rate_upper"] - df_agg["ga_rate"]

# Filter out market segments with large confidence intervals
df_agg = df_agg.query("ga_rate_ub < 1")

# Create a scatter plot of the GoA rate per market segment
fig = px.scatter(
    df_agg.sort_values("ga_rate"),
    y="ga_rate",
    error_y="ga_rate_ub",
    error_y_minus="ga_rate_lb",
    color="n_landings",
    size=[1] * len(df_agg),
    size_max=8,
)
fig.update_layout(
    yaxis_title="GoA rate [/1000 landings]",
    xaxis_title="Market Segment",
    width=width,
    height=height,
    coloraxis_colorbar=dict(title="landings #"),
)
fig.show()
fig.write_image("figures/goa_rate_per_segment.pdf")

### GoA rate per typecode


In [22]:
# Define the aggregation functions
agg_functions = {"GoA": ["mean", "sum"], "flight_id": "count"}

# Copy the landing data and add a new column for the typecode
df_temp = df_landings.copy()
df_temp["typecode"] = df_temp["typecode"].str[:3] + "x"

# Aggregate the data by typecode
df_agg = df_temp.groupby("typecode").agg(agg_functions)

# Rename the columns
df_agg.columns = ["ga_rate", "n_ga", "n_landings"]

# Compute the Clopper-Pearson interval for the GoA rate
alpha = 0.05
n = df_agg["n_landings"]
k = df_agg["n_ga"]
lower_ci, upper_ci = st.beta.interval(1 - alpha, k, n - k + 1)
df_agg["ga_rate_lower"] = lower_ci * 1000
df_agg["ga_rate_upper"] = upper_ci * 1000
df_agg["ga_rate"] = df_agg["ga_rate"] * 1000

# Compute the lower and upper bounds of the confidence interval
df_agg["ga_rate_lb"] = df_agg["ga_rate"] - df_agg["ga_rate_lower"]
df_agg["ga_rate_ub"] = df_agg["ga_rate_upper"] - df_agg["ga_rate"]

# Filter out typecodes with too large confidence intervals
df_agg = df_agg.query("(0 < ga_rate_ub < 1) or (0 < ga_rate_lb < 1)")

# Create a scatter plot of the GoA rate per typecode
fig = px.scatter(
    df_agg.sort_values("ga_rate"),
    y="ga_rate",
    error_y="ga_rate_ub",
    error_y_minus="ga_rate_lb",
    color="n_landings",
    size=[1] * len(df_agg),
    size_max=8,
)
fig.update_layout(
    yaxis_title="GoA rate [/1000 landings]",
    xaxis_title="Typecode",
    width=width,
    height=height,
    coloraxis_colorbar=dict(title="landings #"),
)
fig.write_image("figures/goa_rate_per_typecode.pdf")
fig.show()

### GoA rate per airport


In [23]:
# Define the aggregation functions
agg_functions = {"GoA": ["mean", "sum"], "flight_id": "count"}

# Aggregate the data by airport
df_agg = df_landings.groupby("airport").agg(agg_functions)

# Rename the columns
df_agg.columns = ["ga_rate", "n_ga", "n_landings"]

# Compute the Clopper-Pearson interval for the GoA rate
alpha = 0.05
n = df_agg["n_landings"]
k = df_agg["n_ga"]
lower_ci, upper_ci = st.beta.interval(1 - alpha, k, n - k + 1)
df_agg["ga_rate_lower"] = lower_ci * 1000
df_agg["ga_rate_upper"] = upper_ci * 1000
df_agg["ga_rate"] = df_agg["ga_rate"] * 1000

# Compute the lower and upper bounds of the confidence interval
df_agg["ga_rate_lb"] = df_agg["ga_rate"] - df_agg["ga_rate_lower"]
df_agg["ga_rate_ub"] = df_agg["ga_rate_upper"] - df_agg["ga_rate"]

# Filter out airports with large confidence intervals
df_agg = df_agg.query("ga_rate_ub < 0.5")

# Create a scatter plot of the GoA rate per airport
fig = px.scatter(
    df_agg.sort_values("ga_rate"),
    y="ga_rate",
    error_y="ga_rate_ub",
    error_y_minus="ga_rate_lb",
    color="n_landings",
    size=[1] * len(df_agg),
    size_max=8,
)
fig.update_layout(
    yaxis_title="GoA rate",
    xaxis_title="Airport",
    width=width,
    height=height,
    coloraxis_colorbar=dict(title="andings #"),
)
fig.show()
fig.write_image("figures/goa_rate_per_airport.pdf")

# Define the airport order dictionary
airport_order = {"airports": df_agg.sort_values("ga_rate").index.tolist()}

## Trajectory level


### Loading GoA Trajectories


In [24]:
t_gas = Traffic.from_file(opj("data", f"goas19-23.parquet"))
t_gas = t_gas[df_landings.query("GoA").flight_id.unique()]
t_gas

,count
flight_id,
SWT2KT_6372,8823
NJE936K_61907,7849
DLH3TN_64801,7458
NJE834U_17267,7362
KNE4014_80596,7303
NCG01_39433,6961
NCG03_41843,6886
MSC803_278,6643
NJE741Y_34119,6550


In [58]:
df_goas = df_landings.query("GoA")
list_goa_portions = []
for f in tqdm(t_gas):
    t0, tf = df_goas.query("flight_id==@f.flight_id")["attempt_times"].iloc[0][:2]
    t0 = pd.to_datetime(t0, utc=True)
    tf = pd.to_datetime(tf, utc=True)
    list_goa_portions.append(f.between(t0, tf))
goa_portions = Traffic.from_flights(list_goa_portions)
goa_portions

,count
flight_id,
SWT2KT_6372,7484
KNE4014_80596,6555
MSC803_278,5639
NJE834U_17267,5569
EFD6H_79905,4594
THY4BK_30392,4383
THY7933_20121,4237
PGT1A_80494,4165
THY7VR_26526,4140


### Calculating GoA distance and fuel consumption


In [59]:
def add_goa_metrics(f):
    try:
        # first we check the distance between the two landing attempts
        f.data = f.data.assign(d_start_end=f.distance())
        # we compute the length of the goa portioin
        f = f.cumulative_distance()
        # we compute the fuel consumption of the goa portion assuming 60T of initial mass (valid for a320s)
        f = f.fuelflow(initial_mass=60000)
    except:
        return
    return f


goa_portions = (
    goa_portions.iterate_lazy()
    .pipe(add_goa_metrics)
    .eval(desc="computing distance", max_workers=1)
)

Output()

In [ ]:
# Monkey patching the Flight class
def d_start_end(self):
    return self.data.d_start_end.iloc[0]


Flight.d_start_end = property(d_start_end)

### Agreagating all data in one


In [ ]:
infoGoA = (
    goa_portions.summary(
        [
            "flight_id",
            "callsign",
            "start",
            "stop",
            "destination",
            "cumdist_max",
            "d_start_end",
            "fuel_max",
            "typecode",
        ]
    )
    .eval()
    .sort_values("start")
    .rename(columns={"start": "first", "stop": "second"})
)
infoGoA["ga_time"] = infoGoA["second"] - infoGoA["first"]
infoGoA["ga_time_s"] = infoGoA["ga_time"].dt.total_seconds()
infoGoA["ga_time_m"] = infoGoA["ga_time"] / pd.Timedelta("60s")
infoGoA["year"] = infoGoA["first"].dt.year

# Filter summary
infoGoA

,flight_id,callsign,first,second,destination,cumdist_max,d_start_end,fuel_max,typecode,ga_time,ga_time_s,ga_time_m,year
16231,SIA352_115422,SIA352,2019-01-01 04:38:04+00:00,2019-01-01 04:57:40+00:00,EKCH,58.511732,14.417725,1751.937814,B77W,0 days 00:19:36,1176.0,19.600000,2019
11389,KLM1178_158786,KLM1178,2019-01-01 06:37:18+00:00,2019-01-01 06:58:54+00:00,EHAM,69.790405,20.964207,475.543337,E75L,0 days 00:21:36,1296.0,21.600000,2019
11213,JAI236_195144,JAI236,2019-01-01 06:53:15+00:00,2019-01-01 07:17:18+00:00,EHAM,76.024471,18.539216,1797.928904,A333,0 days 00:24:03,1443.0,24.050000,2019
10962,IBK57F_90596,IBK57F,2019-01-01 09:04:33+00:00,2019-01-01 09:23:22+00:00,EKCH,56.603217,11.405235,655.221193,B738,0 days 00:18:49,1129.0,18.816667,2019
15454,RYR909_44230,RYR909,2019-01-01 11:29:33+00:00,2019-01-01 11:47:38+00:00,EGSS,54.220718,7.128875,657.816947,B738,0 days 00:18:05,1085.0,18.083333,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18810,TVF74PP_14712,TVF74PP,2023-05-30 13:32:03+00:00,2023-05-30 14:12:54+00:00,LFPO,147.975728,71.677855,1207.110488,B738,0 days 00:40:51,2451.0,40.850000,2023
458,AEA7317_30369,AEA7317,2023-05-30 15:33:44+00:00,2023-05-30 16:07:58+00:00,LEMD,133.745548,69.122397,937.924844,B738,0 days 00:34:14,2054.0,34.233333,2023
19946,WZZ74RK_3798,WZZ74RK,2023-05-30 16:06:14+00:00,2023-05-30 16:31:58+00:00,EPWA,97.940375,39.388240,587.777349,A21N,0 days 00:25:44,1544.0,25.733333,2023
17337,TAY7LQ_62152,TAY7LQ,2023-05-30 22:34:00+00:00,2023-05-30 23:06:59+00:00,LFPG,137.035554,52.853480,854.741016,B734,0 days 00:32:59,1979.0,32.983333,2023


In [53]:
infoGoA["speed"] = infoGoA.cumdist_max / infoGoA.ga_time_m * 60

### Filtering GoAs


In [54]:
infoGoA_filtered = infoGoA.query("ga_time_s>200 & (d_start_end<2) & speed<300")
infoGoA_filtered.sort_values("ga_time_s", ascending=True)

,flight_id,callsign,first,second,destination,cumdist_max,d_start_end,fuel_max,typecode,ga_time,ga_time_s,ga_time_m,year,speed
11128,IFA310_4093,IFA310,2021-03-02 11:14:24+00:00,2021-03-02 11:28:00+00:00,EDDB,35.159288,0.754165,NaN,GLEX,0 days 00:13:36,816.0,13.600000,2021,155.114505
12948,OEGCA_12630,OEGCA,2020-06-09 10:40:11+00:00,2020-06-09 10:53:52+00:00,LOWW,33.699026,0.114499,NaN,C56X,0 days 00:13:41,821.0,13.683333,2020,147.766741
11126,IFA309_4088,IFA309,2021-03-01 16:20:04+00:00,2021-03-01 16:34:17+00:00,EDDB,36.734107,0.291526,NaN,GLEX,0 days 00:14:13,853.0,14.216667,2021,155.032572
10461,GDK1_17491,GDK1,2020-12-29 15:17:44+00:00,2020-12-29 15:32:35+00:00,LOWW,37.047449,0.234680,NaN,C68A,0 days 00:14:51,891.0,14.850000,2020,149.686662
13974,QGA017H_734,QGA017H,2020-06-20 15:44:16+00:00,2020-06-20 16:00:16+00:00,EDDB,46.267906,0.492510,NaN,CL30,0 days 00:16:00,960.0,16.000000,2020,173.504647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18923,UAE205_54875,UAE205,2021-07-13 14:23:03+00:00,2021-07-13 16:00:06+00:00,LIMC,438.151571,0.773754,8769.248202,B77W,0 days 01:37:03,5823.0,97.050000,2021,270.881960
12772,NCG03_72464,NCG03,2022-08-04 12:54:38+00:00,2022-08-04 14:32:40+00:00,EHAM,230.983771,0.270159,NaN,D228,0 days 01:38:02,5882.0,98.033333,2022,141.370550
12771,NCG03_72461,NCG03,2022-07-19 12:39:48+00:00,2022-07-19 14:21:34+00:00,EHAM,247.810876,0.408370,NaN,D228,0 days 01:41:46,6106.0,101.766667,2022,146.105331
12767,NCG01_39433,NCG01,2020-12-17 13:09:55+00:00,2020-12-17 15:06:58+00:00,EHAM,263.877424,0.873736,NaN,D228,0 days 01:57:03,7023.0,117.050000,2020,135.263951


In [33]:
infoGoA_filtered.sort_values("cumdist_max", ascending=True)

,flight_id,callsign,first,second,destination,cumdist_max,d_start_end,fuel_max,typecode,ga_time,ga_time_s,ga_time_m,year,speed
12948,OEGCA_12630,OEGCA,2020-06-09 10:40:11+00:00,2020-06-09 10:53:52+00:00,LOWW,33.699026,0.114499,NaN,C56X,0 days 00:13:41,821.0,13.683333,2020,147.766741
11128,IFA310_4093,IFA310,2021-03-02 11:14:24+00:00,2021-03-02 11:28:00+00:00,EDDB,35.159288,0.754165,NaN,GLEX,0 days 00:13:36,816.0,13.600000,2021,155.114505
11126,IFA309_4088,IFA309,2021-03-01 16:20:04+00:00,2021-03-01 16:34:17+00:00,EDDB,36.734107,0.291526,NaN,GLEX,0 days 00:14:13,853.0,14.216667,2021,155.032572
10461,GDK1_17491,GDK1,2020-12-29 15:17:44+00:00,2020-12-29 15:32:35+00:00,LOWW,37.047449,0.234680,NaN,C68A,0 days 00:14:51,891.0,14.850000,2020,149.686662
19949,WZZ803_5636,WZZ803,2020-05-22 10:44:28+00:00,2020-05-22 11:00:58+00:00,EPWA,42.869644,0.546744,637.988227,A321,0 days 00:16:30,990.0,16.500000,2020,155.889615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12771,NCG03_72461,NCG03,2022-07-19 12:39:48+00:00,2022-07-19 14:21:34+00:00,EHAM,247.810876,0.408370,NaN,D228,0 days 01:41:46,6106.0,101.766667,2022,146.105331
12767,NCG01_39433,NCG01,2020-12-17 13:09:55+00:00,2020-12-17 15:06:58+00:00,EHAM,263.877424,0.873736,NaN,D228,0 days 01:57:03,7023.0,117.050000,2020,135.263951
12770,NCG03_41843,NCG03,2021-02-25 11:44:04+00:00,2021-02-25 13:42:37+00:00,EHAM,275.487549,0.663264,NaN,D228,0 days 01:58:33,7113.0,118.550000,2021,139.428536
19800,WZZ2031_122897,WZZ2031,2022-07-29 21:41:32+00:00,2022-07-29 23:00:57+00:00,LEMD,323.924618,1.990671,2343.056829,A320,0 days 01:19:25,4765.0,79.416667,2022,244.727938


In [34]:
# # make 3 subplots (3 columns) and plot line_mapbox in each
# fig =

px.scatter_mapbox(
    t_gas[
        infoGoA_filtered.sort_values("cumdist_max", ascending=True)
        .tail(1)
        .flight_id.unique()
    ]
    .query("geoaltitude<11000")
    .data,
    lat="latitude",
    lon="longitude",
    hover_data=["flight_id", "timestamp"],
    mapbox_style="carto-positron",
    color="altitude",
    height=500,
    width=800,
)

### GoA Distances


In [35]:
# sort box plot by cumdist median
var = "cumdist_max"
category_orders = {
    "destination": infoGoA_filtered.groupby("destination")[[var]]
    .median()
    .sort_values(var)
    .index
}
fig = px.box(
    infoGoA_filtered,
    x="destination",
    y=var,
    color="destination",
    points="outliers",
    category_orders=category_orders,
)
fig.update_yaxes(tickformat=".0f")
fig.update_layout(
    height=height,
    width=width,
    yaxis_type="log",
    xaxis_title="Airport",
    yaxis_title="GoA distance [NM]",
    showlegend=False,
)

fig.write_image("figures/goa_flown_distance.pdf")
fig.show()

In [36]:
def percentile(n):
    def percentile_(x):
        return x.quantile(n)

    percentile_.__name__ = "percentile_{:02.0f}".format(n * 100)
    return percentile_


statistic = (
    infoGoA_filtered.groupby("destination")
    .agg(
        {
            "cumdist_max": [
                "min",
                percentile(0.25),
                "mean",
                "median",
                percentile(0.85),
                "max",
                "count",
            ]
        }
    )
    .sort_values(("cumdist_max", "median"))
)
# round values to 1 decimal
statistic = statistic.round(1)
statistic

cumdist_max                                                       
                    min percentile_25   mean median percentile_85    max count
destination                                                                   
EGSS               70.0          70.7   74.9   72.9          79.7   83.6     4
EDDB               35.2          43.9   91.9   73.5         145.0  239.4     8
LIRF               62.4          72.0   81.6   81.6          95.0  100.8     2
EGCC               66.0          75.9   92.0   81.8         119.7  132.4     6
EDDM               77.7          83.5  105.1   89.4         130.6  148.3     3
LOWW               33.7          71.2   90.6   96.1         130.6  133.7     9
LFPG               99.2          99.2   99.2   99.2          99.2   99.2     1
LSZH               91.2          96.5  106.5  101.9         119.0  126.4     3
EDDF              103.1         103.1  103.1  103.1         103.1  103.1     1
LIMC               84.6          89.2  183.4  105.4         295.0  438.2     4
EIDW               68.0          88.7   99.5  109.5         117.6  121.1     3
LSGG               96.9         108.6  119.4  113.8         133.9  158.6     6
EPWA               42.9          56.6  117.7  120.0         208.7  236.2    16
EHAM              231.0         243.6  254.5  255.8         270.3  275.5     4
LEMD              323.9         323.9  323.9  323.9         323.9  323.9     1

### GoA Times


In [37]:
# sort box plot by cumdist median
var = "ga_time_m"
category_orders = {
    "destination": infoGoA_filtered.groupby("destination")[[var]]
    .median()
    .sort_values(var)
    .index
}
fig = px.box(
    infoGoA_filtered,
    x="destination",
    y=var,
    color="destination",
    points="outliers",
    category_orders=category_orders,
)
fig.update_yaxes(tickformat=".0f")
fig.update_layout(
    height=height,
    width=width,
    yaxis_type="log",
    xaxis_title="Airport",
    yaxis_title="GoA Duration [min]",
    showlegend=False,
)
fig.write_image("figures/goa_time_s.pdf")
fig.show()

In [38]:
statistic = (
    infoGoA_filtered.groupby("destination")
    .agg(
        {
            "ga_time_m": [
                "min",
                percentile(0.25),
                "mean",
                "median",
                percentile(0.85),
                "max",
                "count",
            ]
        }
    )
    .sort_values(("ga_time_m", "median"))
)
# round values to 1 decimal
statistic = statistic.round(1)
statistic

ga_time_m                                                       
                  min percentile_25   mean median percentile_85    max count
destination                                                                 
EDDB             13.6          15.6   29.1   22.5          41.5   65.5     8
EGCC             22.8          23.7   28.7   24.7          36.8   41.1     6
LIRF             21.4          23.5   25.6   25.6          28.5   29.8     2
EGSS             22.8          25.6   26.9   26.9          29.2   30.8     4
LOWW             13.7          22.8   30.1   31.7          38.9   48.0     9
LIMC             24.3          25.9   46.4   32.1          70.4   97.0     4
LSZH             28.0          30.1   38.8   32.2          49.0   56.2     3
EDDF             32.7          32.7   32.7   32.7          32.7   32.7     1
EIDW             23.6          29.1   31.6   34.6          36.0   36.6     3
EPWA             16.5          19.8   35.7   35.0          52.7   67.4    16
LFPG             35.4          35.4   35.4   35.4          35.4   35.4     1
LSGG             28.2          34.2   38.4   37.8          43.7   51.7     6
EDDM             31.9          38.8   42.8   45.6          49.2   50.7     3
LEMD             79.4          79.4   79.4   79.4          79.4   79.4     1
EHAM             98.0         100.8  108.8  109.4         117.9  118.6     4

In [39]:
statistic = (
    infoGoA_filtered.groupby("destination")
    .agg(
        {
            "cumdist_max": [percentile(0.25), "median", percentile(0.85), "max"],
            "ga_time_m": [percentile(0.25), "median", percentile(0.85), "max"],
        }
    )
    .sort_values(("ga_time_m", "median"))
)
# round values to 1 decimal
statistic = statistic.round(1)
statistic

cumdist_max                                 ga_time_m         \
            percentile_25 median percentile_85    max percentile_25 median   
destination                                                                  
EDDB                 43.9   73.5         145.0  239.4          15.6   22.5   
EGCC                 75.9   81.8         119.7  132.4          23.7   24.7   
LIRF                 72.0   81.6          95.0  100.8          23.5   25.6   
EGSS                 70.7   72.9          79.7   83.6          25.6   26.9   
LOWW                 71.2   96.1         130.6  133.7          22.8   31.7   
LIMC                 89.2  105.4         295.0  438.2          25.9   32.1   
LSZH                 96.5  101.9         119.0  126.4          30.1   32.2   
EDDF                103.1  103.1         103.1  103.1          32.7   32.7   
EIDW                 88.7  109.5         117.6  121.1          29.1   34.6   
EPWA                 56.6  120.0         208.7  236.2          19.8   35.0   
LFPG                 99.2   99.2          99.2   99.2          35.4   35.4   
LSGG                108.6  113.8         133.9  158.6          34.2   37.8   
EDDM                 83.5   89.4         130.6  148.3          38.8   45.6   
LEMD                323.9  323.9         323.9  323.9          79.4   79.4   
EHAM                243.6  255.8         270.3  275.5         100.8  109.4   

                                  
            percentile_85    max  
destination                       
EDDB                 41.5   65.5  
EGCC                 36.8   41.1  
LIRF                 28.5   29.8  
EGSS                 29.2   30.8  
LOWW                 38.9   48.0  
LIMC                 70.4   97.0  
LSZH                 49.0   56.2  
EDDF                 32.7   32.7  
EIDW                 36.0   36.6  
EPWA                 52.7   67.4  
LFPG                 35.4   35.4  
LSGG                 43.7   51.7  
EDDM                 49.2   50.7  
LEMD                 79.4   79.4  
EHAM                117.9  118.6

### Fuel Consumption


In [40]:
var = "fuel_max"
category_orders = {
    "destination": infoGoA_filtered.query('typecode=="A320"')
    .groupby("destination")[[var]]
    .median()
    .sort_values(var)
    .index
}
fig = px.box(
    infoGoA_filtered.query('typecode=="A320"'),
    x="destination",
    y=var,
    color="destination",
    points="outliers",
    category_orders=category_orders,
)
fig.update_yaxes(tickformat=".0f")
fig.update_layout(
    height=height,
    width=width,
    yaxis_type="log",
    xaxis_title="Airport",
    yaxis_title="GoA Fuel Consumption [Kg]",
    showlegend=False,
)
# save fig pdf
fig.write_image("figures/a320_goa_fuel.pdf")
fig.show()

In [52]:
infoGoA_filtered

,flight_id,callsign,first,second,destination,cumdist_max,d_start_end,fuel_max,typecode,ga_time,ga_time_s,ga_time_m,year,speed
11149,ISS7021_4070,ISS7021,2019-02-06 10:52:03+00:00,2019-02-06 11:18:25+00:00,LIMC,90.700599,0.279897,1856.996256,B763,0 days 00:26:22,1582.0,26.366667,2019,206.398329
11150,ISS7021_4071,ISS7021,2019-02-20 09:47:39+00:00,2019-02-20 10:11:58+00:00,LIMC,84.555244,0.619226,1793.718926,B763,0 days 00:24:19,1459.0,24.316667,2019,208.635283
7868,DLH77X_127043,DLH77X,2019-04-30 06:27:05+00:00,2019-04-30 07:02:28+00:00,LFPG,99.157639,1.986781,794.624242,A20N,0 days 00:35:23,2123.0,35.383333,2019,168.142958
16398,SWR115U_68373,SWR115U,2019-05-04 09:05:04+00:00,2019-05-04 09:33:01+00:00,LSZH,91.184356,0.978542,NaN,BCS3,0 days 00:27:57,1677.0,27.950000,2019,195.744593
12105,LDM50_15322,LDM50,2020-03-22 11:04:10+00:00,2020-03-22 11:40:15+00:00,LOWW,131.536296,1.559900,1128.341883,A320,0 days 00:36:05,2165.0,36.083333,2020,218.720861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16250,SPMXI_59830,SPMXI,2022-02-12 20:24:21+00:00,2022-02-12 21:10:40+00:00,EPWA,135.377492,1.581074,NaN,P180,0 days 00:46:19,2779.0,46.316667,2022,175.372066
12586,MMTOO_25066,MMTOO,2022-04-08 11:59:49+00:00,2022-04-08 12:41:37+00:00,EDDB,148.334412,1.157193,NaN,CL30,0 days 00:41:48,2508.0,41.800000,2022,212.920208
12771,NCG03_72461,NCG03,2022-07-19 12:39:48+00:00,2022-07-19 14:21:34+00:00,EHAM,247.810876,0.408370,NaN,D228,0 days 01:41:46,6106.0,101.766667,2022,146.105331
19800,WZZ2031_122897,WZZ2031,2022-07-29 21:41:32+00:00,2022-07-29 23:00:57+00:00,LEMD,323.924618,1.990671,2343.056829,A320,0 days 01:19:25,4765.0,79.416667,2022,244.727938


In [42]:
# objective, find when the aircraft initialise the go around
def get_a320_portions(f):
    try:
        first, second, *_ = f.aligned_on_ils(f.destination, angle_tolerance=0.15)
        # # find where the altitude is minimum
        idx = first.data.geoaltitude.idxmin()
        t0, d0 = first.data.loc[idx, ["timestamp", "distance"]]
        tf = second.query(f"distance>{d0}").data.timestamp.iloc[-1]
        # portion = f.between(t0, tf)
        portion = f.after(t0)
    except:
        return
    return portion.fuelflow(initial_mass=60000)


a320_portions = (
    t_gas[infoGoA_filtered.query('typecode=="A320"').flight_id.unique()]
    .iterate_lazy()
    .pipe(get_a320_portions)
    .eval(desc="computing fuel", max_workers=1)
)
a320_portions

Output()

In [43]:
infoA320 = (
    a320_portions.summary(
        [
            "flight_id",
            "callsign",
            "start",
            "stop",
            "destination",
            "fuel_max",
            "typecode",
        ]
    )
    .eval()
    .sort_values("start")
)
infoA320

AttributeError: 'NoneType' object has no attribute 'summary'

In [ ]:
fig = px.box(
    infoA320,
    x="destination",
    y="fuel_max",
    color="destination",
    category_orders={
        "destination": infoA320.groupby("destination")[["fuel_max"]]
        .median()
        .sort_values("fuel_max")
        .index
    },
)
fig.update_layout(
    height=height,
    width=width,
    yaxis_type="log",
    xaxis_title="Airport",
    yaxis_title="GoA Fuel Consumption [Kg]",
    showlegend=False,
)
fig.write_image("figures/a320_goa_fuel.pdf")
fig.show()

## Impact on other landing traffic


### Finding ASMA Sector


In [44]:
def angular_difference(b1, b2):
    return 180 - np.abs(np.abs(b1 - b2) - 180)


def add_asma_sector(group):
    # Fit KDE to bearing data
    kde = st.gaussian_kde(group.C40_BEARING, bw_method=0.08)
    # Find all local maxima of the KDE
    maxima_indices = argrelextrema(kde.pdf(np.linspace(0, 360, 360 + 1)), np.greater)

    # Get the x values corresponding to the maxima indices
    modes = np.linspace(0, 360, 360 + 1)[maxima_indices]

    # sort the modes
    modes = np.sort(modes)

    # associate each data point of group.C40_BEARING to the closest mode
    # compute the distance between each data point and each mode
    distances = np.abs(np.subtract.outer(group.C40_BEARING.values, modes))
    # associate each data point to the closest mode
    group["ASMA_sector"] = np.argmin(distances, axis=1)

    # if there is lest than 15 deg difference between the first and last mode, merge them on the mean
    if angular_difference(modes[0], modes[-1]) < 15:
        group.loc[group["ASMA_sector"] == len(modes) - 1, "ASMA_sector"] = 0
    group["ASMA_sector"] = group["ASMA_sector"].astype(str)
    return group


groups = []
for (airport, runway), group in tqdm(df_landings.groupby(["airport", "AP_C_RWY"])):
    if len(group) < 1000:
        continue
    group = group.dropna(subset=["C40_BEARING"])
    group = add_asma_sector(group)
    groups.append(group)

df_landings_with_asma = pd.concat(groups)

  0%|          | 0/136 [00:00<?, ?it/s]

In [45]:
import plotly.graph_objects as go

# Filter data
selected_airport = "EDDF"
selected_runway = "25L"
subset = (
    df_landings_with_asma.query(
        "airport==@selected_airport & AP_C_RWY==@selected_runway"
    )
    .sample(10000)
    .sort_values("ASMA_sector")
)

fig1 = px.scatter_mapbox(
    subset,
    lat="C40_CROSS_LAT",
    lon="C40_CROSS_LON",
    color="ASMA_sector",
    mapbox_style="carto-positron",
    zoom=8,
    height=800,
    width=800,
)
# update map center to the airport
fig1.update_layout(
    mapbox_center={
        "lat": airports[selected_airport].centroid.coords[0][1],
        "lon": airports[selected_airport].centroid.coords[0][0],
    },
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

fig1.update_layout(showlegend=False)

#
fig3 = px.histogram(
    subset, x="C40_BEARING", color="ASMA_sector", height=500, width=500, nbins=100
)
kde = st.gaussian_kde(subset.C40_BEARING, bw_method=0.08)
x = np.linspace(0, 360, 1000)
fig3.add_trace(
    go.Scatter(
        x=x,
        y=kde.pdf(x)
        * np.histogram(subset.C40_BEARING, bins=100)[0].max()
        / np.max(kde.pdf(x)),
        name="kde",
        mode="lines",
        line=dict(color="black", dash="dash", width=0.5),
    )
)
fig3.update_layout(
    xaxis_title="ASMA entry bearing [deg]",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
    ),
)
fig1.show()
fig3.show()
fig1.write_image("figures/asma_map2sector.pdf")
fig3.write_image("figures/asma_bearing2sector.pdf")

In [46]:
tqdm.pandas()
import numpy as np

# Define the columns to group by
group_cols = ["airport", "ASMA_sector", "AP_C_RWY", "AC_CLASS"]

# Calculate the time difference between stop time and C40 cross time
df_landings_with_asma["time40"] = (
    df_landings_with_asma["landing_time"] - df_landings_with_asma["C40_CROSS_TIME"]
)

# Calculate the 20th percentile of time40 for each group of columns where GoA is False
dict_ref_times = (
    df_landings_with_asma.query("not GoA")
    .groupby(group_cols)
    .progress_apply(lambda x: x["time40"].quantile(q=0.2))
    .to_dict()
)

df_landings_with_asma["ref_time"] = df_landings_with_asma.progress_apply(
    lambda x: dict_ref_times.get(
        (x.airport, x.ASMA_sector, x.AP_C_RWY, x.AC_CLASS), np.nan
    ),
    axis=1,
)
# # Drop rows with missing values in the group columns
# df_landings_with_asma = df_landings_with_asma.dropna(subset=group_cols)

# # Calculate the extra time for each row by subtracting the reference time for the group
df_landings_with_asma["extra_time"] = (
    df_landings_with_asma["time40"] - df_landings_with_asma["ref_time"]
)

# # Convert the extra time to seconds
df_landings_with_asma["extra_time_s"] = df_landings_with_asma[
    "extra_time"
].dt.total_seconds()

  0%|          | 0/2183 [00:00<?, ?it/s]

  0%|          | 0/6037446 [00:00<?, ?it/s]

In [48]:
from collections import defaultdict

airport2windows = {}

# Create a new dictionary for the airport in the 'w_results' dictionary
# airport2windows = {}
for airport, airport_group in df_landings_with_asma.groupby("airport"):
    if airport in airport2windows:
        continue
    res_windows = defaultdict(list)

    # Retrieve the times of first GoA Attempt
    time_gas = airport_group.query("GoA").attempt_times.apply(lambda x: x[0])
    window_minutes = 10
    for t_goa in pd.to_datetime(time_gas, utc=True):
        # Compute the reference time as the median extra time in the 20 minutes before the GoA
        t_before = t_goa - timedelta(minutes=20)
        subset_before = airport_group.query("@t_before < landing_time < @t_goa")

        # Skip the current GoA if there is a GoA 20min before or an hour after
        thour = t_goa + timedelta(hours=1)
        subset_next_hour = airport_group.query(f"@t_goa < landing_time < @thour")
        if (
            len(subset_next_hour.query("GoA")) > 1
            or len(subset_before.query("GoA")) > 0
        ):
            continue

        reference_time = subset_before.extra_time_s.median()
        for w0 in range(51):
            # Extract all landings happening dm minutes before and after the GoA
            tw0 = t_goa + timedelta(minutes=w0)
            twf = tw0 + timedelta(minutes=(window_minutes))
            subset_after = airport_group.query(
                "(@tw0 < landing_time < @twf) and (not GoA)"
            )
            additional_time = subset_after.extra_time_s.median() - reference_time
            # if additional_time is nan raise errer
            if np.isnan(additional_time):
                continue
                # raise ValueError(f"additional_time is nan for {t0} {tf} {w0} {window_minutes}")
            res_windows[w0].append(additional_time)
    airport2windows[airport] = {
        k: np.nanmedian(v) for k, v in res_windows.items() if len(v) > 0
    }
    print(airport, airport2windows[airport])

EDDB {0: 31.0, 1: 30.5, 2: 29.25, 3: 28.75, 4: 31.5, 5: 30.5, 6: 39.5, 7: 47.75, 8: 63.25, 9: 63.5, 10: 61.19999999999999, 12: 61.19999999999999, 13: 62.099999999999994, 14: 55.0, 15: 53.5, 16: 41.0, 17: 29.5, 18: 27.5, 19: 25.0, 20: 24.5, 21: 9.5, 22: 11.75, 23: 6.5, 24: -0.5, 25: 3.0, 26: 6.75, 27: 0.75, 28: 6.5, 29: 1.25, 31: 9.0, 32: 9.0, 33: 10.25, 34: 12.75, 35: 9.5, 36: 12.0, 37: 7.75, 38: 6.0, 39: 17.5, 40: 8.5, 44: 7.0, 45: 4.2, 46: -5.0, 47: -3.0, 48: -2.0, 49: -5.0, 50: -5.0, 11: 64.0, 30: 2.0, 41: 7.0, 42: 7.0, 43: 6.5}
EDDF {2: 14.0, 3: 14.0, 4: 12.25, 5: 9.8, 6: 14.25, 7: 15.700000000000003, 8: 19.25, 9: 25.0, 10: 23.5, 11: 26.5, 12: 31.0, 13: 29.0, 14: 27.5, 15: 26.0, 16: 24.0, 17: 22.5, 31: 0.9000000000000057, 32: -4.5, 33: -6.25, 34: -9.0, 35: -6.25, 36: -11.0, 37: -11.25, 38: -8.5, 39: -10.0, 40: -12.0, 41: -13.0, 42: -11.25, 43: -12.5, 44: -11.9, 45: -10.0, 46: -5.5, 47: -2.1000000000000014, 0: 13.3, 1: 13.0, 30: 1.0, 48: -7.75, 49: -10.75, 50: -7.549999999999997, 18

In [49]:
from plotly.subplots import make_subplots

df = pd.DataFrame.from_dict(airport2windows, orient="index")
df = df[sorted(df.columns)]
# replace 'min' in column names by m
df.columns = [f"{c}-{c+window_minutes} min" for c in df.columns]
fig = make_subplots(
    cols=2, rows=10, shared_xaxes=True, vertical_spacing=0.01, horizontal_spacing=0.01
)

colors = px.colors.qualitative.Dark24[:20]
for i, (airport, row) in enumerate(df.iterrows()):
    plotcol = i % 2 + 1
    plotrow = i // 2 + 1
    annotation_y = i + 1
    if plotcol == 2:
        plotrow += 1
        annotation_y = i + 2

    fig.add_trace(
        go.Scatter(
            x=row.index,
            y=row.values,
            name=airport,
            mode="lines+markers",
            marker=dict(color=colors[i]),
        ),
        col=plotcol,
        row=plotrow,
    )
    fig.add_annotation(
        xref="paper",
        yref=f"y{annotation_y}",
        x=plotcol / 2 - 0.05,
        y=50,
        text=airport,
        showarrow=False,
        font=dict(size=15, color=colors[i]),
    )
    fig.update_yaxes(tickvals=[0, 100, 200], row=plotrow, col=plotcol)

fig.update_yaxes(range=[df.values.min(), df.values.max()])
fig.update_layout(
    height=29.7 / 2.54 * 96,
    width=21 / 2.54 * 96,
    showlegend=False,
    margin=dict(t=0, l=100, r=0, b=100),
)

# only display y axis ticks on the first column
for i in range(10):
    fig.update_yaxes(tickvals=[0, 50, 100], ticktext=["", "", ""], row=i + 1, col=2)
    fig.update_yaxes(tickvals=[0, 50, 100], row=i + 1, col=1)


# using anotation add y title to the left of the first column
fig.add_annotation(
    xref="x domain",
    yref="paper",
    x=-0.1,
    y=0.5,
    xanchor="right",
    text="Median ASMA additional time [s]",
    showarrow=False,
    font=dict(size=15),
    textangle=-90,
)

fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0.5,
    y=-0.07,
    yanchor="top",
    text="Time window after GoA",
    showarrow=False,
    font=dict(size=15),
)

fig.show()
fig.write_html("figures/extra_time.html")
fig.write_image("figures/extra_time.pdf")